In [1]:
import os

os.chdir("../")
%pwd

'/home/siddhu/Desktop/Movie-Recommendation-System'

In [ ]:
import zipfile
import gdown
from dataclasses import dataclass
from src.movieRecommendation.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.movieRecommendation.utils.common import read_yaml, create_directories
from src.movieRecommendation.logging import logger

In [3]:
@dataclass
class DataIngestionConfig:
    root_dir: str
    drive_file_id: str
    local_data_file: str
    unzip_dir: str

In [4]:
class ConfigurationManager:
    
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            drive_file_id=config.drive_file_id,  
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

In [5]:
class DataIngestion:
    def __init__(self, config: "DataIngestionConfig"):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info("Downloading dataset from Google Drive...")
            url = f"https://drive.google.com/uc?id={self.config.drive_file_id}"
            gdown.download(url, self.config.local_data_file, quiet=False)
            logger.info("File downloaded successfully!")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}. Skipping download.")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            logger.info(f"Extracting dataset to {unzip_path}...")
            zip_ref.extractall(unzip_path)
            logger.info("Extraction complete.")

In [6]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_data()
data_ingestion.extract_zip_file()

[2026-02-13 13:36:33,966: INFO: common: YAML file 'config/config.yaml' read successfully.]
[2026-02-13 13:36:33,967: INFO: common: YAML file 'params.yaml' read successfully.]
[2026-02-13 13:36:33,968: INFO: common: Directory 'artifacts' created successfully or already exists.]
[2026-02-13 13:36:33,968: INFO: common: Directory 'artifacts/data_ingestion' created successfully or already exists.]
[2026-02-13 13:36:33,969: INFO: 3132787188: Downloading dataset from Google Drive...]


Downloading...
From: https://drive.google.com/uc?id=1PMwyTlpKh-1IThTmTHRYxd1pjIsn5HYd
To: /home/siddhu/Desktop/Movie-Recommendation-System/artifacts/data_ingestion/final-movie-data.zip
100%|██████████| 13.7M/13.7M [00:11<00:00, 1.19MB/s]

[2026-02-13 13:36:49,098: INFO: 3132787188: File downloaded successfully!]
[2026-02-13 13:36:49,100: INFO: 3132787188: Extracting dataset to artifacts/data_ingestion...]


[2026-02-13 13:36:49,259: INFO: 3132787188: Extraction complete.]
